In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
from datetime import date
import operator 
from scipy.optimize import curve_fit
from scipy.optimize import fsolve

from scipy.integrate import odeint
from sympy import integrate, init_printing
from sympy.abc import x
init_printing(use_latex="mathjax")
from sympy import cos,pi

#from covidmx import CovidMX
plt.style.use('fivethirtyeight')
%matplotlib inline 

In [2]:
##MOdelos
def exponential_model(x,a,b):
    return (a/b)*(np.exp(x*b)-1)

def gompertz_model(x,g):
    return k*np.exp(-n0*np.exp(-g*x))

def gompertz_hubbert(x,g):
    return g*n0*np.exp(-g*x)*k*np.exp(-n0*np.exp(-g*x))


# def logistic_model(x,a,b,c):
#     return c/(1+np.exp(-(x-b)/a))

def bertalanffy_model(x,a,b,c):
    return a*(1-np.exp(-b*(x-x0)))**c
    

def logistic_model(x,r,C0,K):
    return K/(1+((K-C0)/C0)*np.exp(-r*x))

def hubbert_curve(x,r,C0,K):
    return (np.exp(r*x)*r*C0*(K-C0))/(K*(1+C0/K *(np.exp(r*x) -1))**2)

def SIQR_COVID(y,t,N,beta,gamma,alpha,alpha_3,eta):
    S, I, Q, R = y
    dSdt = -beta*S*I/N 
    dIdt = beta*S*I/N - (alpha+ eta)*I
    dQdt = eta*I - gamma*Q
    dRdt = gamma*Q
    return dSdt, dIdt, dQdt, dRdt
def SIQR_1(y,t,Lamda,d,xi,gamma,delta,alpha1,epsilon,alpha2,b0,b1,phi):
    S, I, Q, R = y
    dSdt = Lamda-b0*(1 + b1*np.cos(2*np.pi*t+phi))*S *I - d*S+xi*R
    dIdt = b0*(1 + b1*np.cos(2*np.pi*t+phi))*S *I - (alpha1 + gamma + delta+ d)*I
    dQdt = delta *I - (epsilon + d+ alpha2)*Q 
    dRdt = epsilon *Q + gamma*I - (d+xi)*R
    return dSdt, dIdt, dQdt, dRdt
def SIQR_2(y,t,Lamda,d,xi,gamma,delta,alpha1,epsilon,alpha2,b0,b1,phi,k):
    S, I, Q, R = y
    dSdt = Lamda-b0*(1 + b1*np.cos(2*np.pi*t+phi))*S *I/(1+k*I) - d*S+xi*R
    dIdt = b0*(1 + b1*np.cos(2*np.pi*t+phi))*S *I/(1+k*I) - (alpha1 + gamma + delta+ d)*I
    dQdt = delta *I - (epsilon + d+ alpha2)*Q 
    dRdt = epsilon *Q + gamma*I - (d+xi)*R
    return dSdt, dIdt, dQdt, dRdt


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        plt.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',rotation=90, size=8)
def SIRm(y,t,beta1,lamda,phi_0,mu,d,a,w,nu):
    S, I, M = y
    dSdt = mu*(1-phi_0) -mu* (d*M)/(d*M+1) -  (beta1 + lamda*np.cos(2*np.pi*t))*S * I  - mu*S
    dIdt =  (beta1 + lamda*np.cos(2*np.pi*t))*S * I - (mu + nu)* I
    dMdt = a*w*I-a*M
    return dSdt, dIdt, dMdt
def SISm(y,t,beta1,lamda,phi_0,mu,d,a,w,nu,epsilon_1):
    S, I, M = y
    dSdt = mu*(1-phi_0) -mu*(1-phi_0-epsilon_1)* (d*M)/(d*M+1) -  (beta1 + lamda* np.cos(2*np.pi*t))*S * I  - mu*S +nu*I
    dIdt =  (beta1 + lamda*np.cos(2*np.pi*t))*S * I - (mu + nu)* I
    dMdt = a*w*I-a*M
    return dSdt, dIdt, dMdt

In [ ]:
def plot_SISm(t,S,I,M):
    plt.figure(figsize=(16, 9))
    
    plt.plot(t,S,label='Susceptible')
    plt.plot(t,I,label='Infectados')
    #plt.plot(t,M)
    plt.xlim(0,30)
    plt.xlabel('Tiempo en semanas', size=30)
    plt.ylabel('Fracción de la población', size=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()
    
    plt.figure(figsize=(16, 9))
    #plt.plot(t,S,label='Susceptible')
    #plt.plot(t,I,label='Infectados')
    plt.plot(t,M,label='M')
    plt.xlim(0,30)
    plt.ylim(0,0.1)
    plt.xlabel('Tiempo en semanas', size=30)
    plt.ylabel('índice de información', size=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()
    

In [ ]:
def plot_SISm_1(t,S,I,M):
    plt.figure(figsize=(20, 15))
    plt.subplot(3,1,1)
    plt.plot(t,S,label='Susceptible')
    #plt.plot(t,I,label='Infectados')
    #plt.plot(t,M)
    plt.xlim(0,30)
    plt.xlabel('Tiempo en semanas', size=20)
    plt.ylabel('Fracción de la población', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    
    plt.subplot(3,1,2)
    plt.plot(t,I,label='Infectados')
    plt.xlim(0,30)
    plt.xlabel('Tiempo en semanas', size=20)
    plt.ylabel('Fracción de la población', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)

    
    
    plt.subplot(3,1,3)
    plt.plot(t,M,label='M')
    plt.xlim(0,30)
    plt.ylim(0,0.1)
    plt.xlabel('Tiempo en semanas', size=20)
    plt.ylabel('índice de información', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.savefig("E:\\Graficas\\SISM1.pdf", dpi=150, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_SIQRS1(t,S1,I1,Q1,R1,S2, I2, Q2, R2 ):
    plt.figure(figsize=(16, 9))
    plt.plot(t,S1,label='SIQRS')
    plt.plot(t,S2,label='SIRS',linestyle='dashed')
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Numeros de individuos suceptibles', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()
    
    plt.figure(figsize=(16, 9))
    plt.plot(t,I1,label='SIQRS')
    plt.plot(t,I2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Numeros de individuos infectados', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()
    
    plt.figure(figsize=(16, 9))
    plt.plot(t,Q1,label='SIQRS')
    plt.plot(t,Q2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Numeros de individuos aislamiento', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()
    
    plt.figure(figsize=(16, 9))
    plt.plot(t,R1,label='SIQRS')
    plt.plot(t,R2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Numeros de individuos Recuperados', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.show()

In [ ]:
def plot_SIQRS1_1(t,S1,I1,Q1,R1,S2, I2, Q2, R2 ):
    plt.figure(figsize=(20, 20))
    plt.subplot(2,2,1)
    
    plt.plot(t,S1,label='SIQRS')
    plt.plot(t,S2,label='SIRS',linestyle='dashed')
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Números de individuos susceptibles', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    #plt.show()
    
    plt.subplot(2,2,2)
    #plt.figure(figsize=(16, 9))
    plt.plot(t,I1,label='SIQRS')
    plt.plot(t,I2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Números de individuos infeciosos', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    #plt.show()
    
    plt.subplot(2,2,3)
    #plt.figure(figsize=(16, 9))
    plt.plot(t,Q1,label='SIQRS')
    plt.plot(t,Q2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Números de individuos en aislamiento', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    #plt.show()
    
    plt.subplot(2,2,4)
    #plt.figure(figsize=(16, 9))
    plt.plot(t,R1,label='SIQRS')
    plt.plot(t,R2,label='SIRS',linestyle='dashed')
    #plt.plot(t,M)
    #plt.xlim(0,30)
    plt.xlabel('Tiempo en años', size=20)
    plt.ylabel('Números de individuos Recuperados', size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    legend = plt.legend(fontsize=20)
    plt.savefig("E:\\Graficas\\SIQRS1.pdf", dpi=150, transparent=True, bbox_inches='tight')
    #plt.savefig("E:\\Graficas\\SIQRS1.eps", dpi=150, bbox_inches='tight', transparent=True)
    #plt.grid()
    plt.show()

In [ ]:
#modelo SIS con vacunacion 
parametrosSISm=(2.295,-1.8,0.75,1/(58*75),1200/(1-0.75-0.01),0.9,0.1,0.5,0.01)
incialesSISm=0.8,0.001,0.001*0.1
#integracion
t = np.linspace(0, 150,15000)
ret = odeint(SISm, incialesSISm, t, args=parametrosSISm)
S, I, M = ret.T

In [ ]:
#SIQRS ejemplo 1
Lamda,d,xi,gamma,delta,alpha1,epsilon,alpha2,b0,b1,phi= 3.3852,0.0013,1.8,36,7,0,2,0,0.0169,0.36,0.6
parametrosSIQRS1=(Lamda,d,xi,gamma,delta,alpha1,epsilon,alpha2,b0,b1,phi)
parametrosSIRS1=(Lamda,d,xi,gamma,0,alpha1,0,alpha2,b0,b1,phi)
incialesSIQRS1=2554,50,0,0

In [ ]:
t = np.linspace(0, 15,15000)
ret1 = odeint(SIQR_1, incialesSIQRS1, t, args=parametrosSIQRS1)
S1, I1, Q1, R1 = ret1.T
ret2 = odeint(SIQR_1, incialesSIQRS1, t, args=parametrosSIRS1)
S2, I2, Q2, R2 = ret2.T

In [ ]:
#condiciones
#
beta_t1=b0*(1+ b1*cos(2*pi*x+phi))
beta_t1
betabarra1=integrate(beta_t1,(x,0,1))
betabarra1
R0_1=(betabarra1*Lamda)/(d*(gamma+delta+d+alpha1))

#condincion2= nu +a*w-a
#condicion3=
R0_1

In [ ]:
plot_SIQRS1_1(t,S1, I1, Q1, R1, S2, I2, Q2, R2)

In [ ]:
#SIQRS ejemplo 2
Lamda,d,xi,gamma=0,0.0013,1.8,36
delta,alpha1,epsilon,alpha2=4,0,1,0
b0,b1,phi, k= 44,0.36,0.6,0.048
parametrosSIQRS2=(Lamda,d,xi,gamma,delta,alpha1,epsilon,alpha2,b0,b1,phi,k)
parametrosSIRS2=(Lamda,d,xi,gamma,0,alpha1,0,alpha2,b0,b1,phi,k)
incialesSIQRS2=0.9808,0.0192,0,0

In [ ]:
t = np.linspace(0, 15,15000)
ret1 = odeint(SIQR_2, incialesSIQRS2, t, args=parametrosSIQRS2)
S1, I1, Q1, R1 = ret1.T
ret2 = odeint(SIQR_2, incialesSIQRS2, t, args=parametrosSIRS2)
S2, I2, Q2, R2 = ret2.T

In [ ]:
plot_SIQRS1_1(t,S1, I1, Q1, R1, S2, I2, Q2, R2)